In [1]:
import pandas as pd

df_tiny = pd.read_pickle('out/DataFrame-Generated-2022-05-12.pkl')
df_linx = pd.read_pickle('out/linx.pkl')
# Montagem DataFrame
df = pd.concat([df_tiny, df_linx], axis=0, ignore_index=True)

# Limpeza DataFrame
df.drop(columns=['v_frete_avg'], inplace=True)
df.drop_duplicates(inplace=True)
# Normalização dos dados para Str, já que o Df possui tipos de objetos misturados na mesma coluna.
for col in df.columns:
    df[col] = df[col].astype(str)

df.uf = df.uf.astype('category')
df.xml_version = df.xml_version.astype('category')
df.tipo_comprador = df.tipo_comprador.astype('category')
df.loc[df['tipo_comprador'] == 'idEstrangeiro', 'tipo_comprador'] = 'CPF'

# Ajustes das datas
## datas que tem "T"
array_T = df.loc[df['dt_emissao'].str.contains('T', na=False), 'dt_emissao'].str.split('T').str[0]
df.loc[df['dt_emissao'].str.contains('T', na=False), 'dt_emissao'] = array_T
## Datas com espaços
array_space = df.loc[df['dt_emissao'].str.contains(r'\s', na=False), 'dt_emissao'].str.split(r'\s').str[0]
df.loc[df['dt_emissao'].str.contains(r'\s', na=False), 'dt_emissao'] = array_space

df['dt_emissao'] = pd.to_datetime(df['dt_emissao'])
df.sort_values(by='dt_emissao', inplace=True)
df['nome_prod'] = df['nome_prod'].str.title()
# Ajuste de Col QTD
df = df.loc[df['qtd'] != 'Pc']
df['qtd'] = df['qtd'].str.split('\.').str[0].astype(int)
df.num_nfe = df.num_nfe.astype(int)

# Ajuste de Valores
df = df.loc[(df['vl_unit'] != "UN") & (df['vl_unit'] != "Un") ]
df = df.loc[(df['sub_total'] != "UN") & (df['sub_total'] != "Un") ]
df[['vl_unit', 'sub_total']] = df[['vl_unit', 'sub_total']].replace({'\.':'', ',':''}, regex=True).astype(float) / 100
df['sub_total'] = df['vl_unit'] * df['qtd']
df.reset_index(inplace=True, drop=True)

df.to_pickle('out/Final_df.pkl')